In [1]:
import tkinter as tk
from tkinter import ttk, PhotoImage, messagebox


class MedicalExpertSystemChatbot:
    def __init__(self, root):
        self.root = root
        self.root.title("Medical Expert System Chatbot")
        self.root.geometry("500x600")
        self.root.configure(bg="black")

        # Create the chat window
        self.chat_frame = tk.Frame(root, bg="#212121")
        self.chat_frame.pack(expand=True, fill="both", padx=10, pady=10)

        # Create a canvas for the chat messages
        self.canvas = tk.Canvas(self.chat_frame, bg="#212121", highlightthickness=0)
        self.scrollable_frame = tk.Frame(self.canvas, bg="#212121")

        # Configure the scrollbar
        self.scrollbar = ttk.Scrollbar(self.chat_frame, orient="vertical", command=self.canvas.yview)
        self.canvas.configure(yscrollcommand=self.scrollbar.set)

        self.scrollable_frame.bind("<Configure>", lambda e: self.canvas.configure(scrollregion=self.canvas.bbox("all")))
        self.canvas.create_window((0, 0), window=self.scrollable_frame, anchor="nw")

        # Pack the canvas and scrollbar
        self.canvas.pack(side="left", fill="both", expand=True)
        self.scrollbar.pack(side="right", fill="y")

        # Add the message input field
        self.user_input = tk.Entry(root, width=35, font=("calibri", 14), bg="#4f4f4f", fg="white", borderwidth=0)
        self.user_input.pack(pady=10, padx=10, side=tk.LEFT, expand=True)

        # Send button
        tk.Button(root, text="Send", command=self.process_input, font=("calibri", 12), bg="#4CAF50", fg="white").pack(pady=10, padx=5, side=tk.LEFT)
        
        # Restart button
        self.restart_button = tk.Button(root, text="Restart", command=self.restart, font=("calibri", 12), bg="#f44336", fg="white", state=tk.DISABLED)
        self.restart_button.pack(pady=10, padx=5, side=tk.LEFT)

        # Load icons
        try:
            self.bot_icon = PhotoImage(file="C:/Users/ASUS/Downloads/robot.png").subsample(12, 12)
            self.user_icon = PhotoImage(file="C:/Users/ASUS/Downloads/profile.png").subsample(12, 12)
        except Exception as e:
            self.bot_icon = self.user_icon = None
            messagebox.showwarning("Warning", "Icons not found. Proceeding without icons.")

        # Initialize chatbot logic
        self.symptoms = {}
        self.current_question = 0
        self.symptom_keys = ['fever', 'cough', 'sore_throat', 'body_ache', 'rash', 'headache']
        self.questions = [
            "Do you have a fever?", "Do you have a cough?", "Do you have a sore throat?",
            "Do you have body aches?", "Do you have a rash?", "Do you have a headache?"
        ]
        self.rules = {
            "flu": {"fever": True, "cough": True, "sore_throat": True, "body_ache": True, "rash": False, "headache": False},
            "dengue": {"fever": True, "cough": False, "sore_throat": False, "body_ache": True, "rash": True, "headache": False},
            "tension_headache": {"fever": False, "cough": False, "sore_throat": False, "body_ache": False, "rash": False, "headache": True},
            "minor_throat_infection": {"fever": False, "cough": False, "sore_throat": True, "body_ache": False, "rash": False, "headache": False}
        }

        # Greet the user
        self.display_message("Welcome to the Medical Diagnosis Chatbot!\nPlease answer the following questions.", "bot")
        self.ask_question()

    def display_message(self, message, sender):
        """Display a message in the chat area with icon and bubble."""
        message_frame = tk.Frame(self.scrollable_frame, bg="#212121")

        if sender == "bot":
            if self.bot_icon:
                tk.Label(message_frame, image=self.bot_icon, bg="#212121").pack(side=tk.LEFT, padx=5)

            tk.Label(message_frame, text=message, bg="#4f4f4f", fg="white", font=("calibri", 13), wraplength=350,
                     padx=10, pady=8, justify=tk.LEFT, relief=tk.SOLID,borderwidth=0).pack(side=tk.LEFT)
        else:
            if self.user_icon:
                tk.Label(message_frame, image=self.user_icon, bg="#212121").pack(side=tk.RIGHT, padx=2)

            tk.Label(message_frame, text=message, bg="#4f4f4f", fg="white", font=("calibri", 13), wraplength=300,
                     padx=10, pady=8, justify=tk.RIGHT, relief=tk.SOLID,borderwidth=0).pack(side=tk.RIGHT)

        message_frame.pack(anchor="w" if sender == "bot" else "e", pady=5)
        self.canvas.update_idletasks()
        self.canvas.yview_moveto(1)

    def process_input(self):
        """Process user input and advance to the next question or diagnosis."""
        user_response = self.user_input.get().lower().strip()
        if user_response not in ['yes', 'no']:
            self.display_message("Please respond with 'yes' or 'no'.", "bot")
            return

        self.display_message(user_response, "user")
        self.symptoms[self.symptom_keys[self.current_question]] = (user_response == 'yes')
        self.current_question += 1
        self.user_input.delete(0, tk.END)

        if self.current_question < len(self.questions):
            self.ask_question()
        else:
            self.diagnose()

    def ask_question(self):
        """Ask the current question from the list."""
        question = self.questions[self.current_question]
        self.display_message(question, "bot")

    def diagnose(self):
        """Provide a diagnosis based on the user's symptoms."""
        self.display_message("Diagnosing your symptoms...", "bot")
        for condition, rules in self.rules.items():
            if all(self.symptoms.get(symptom) == rules.get(symptom, False) for symptom in self.symptom_keys):
                self.display_message(f"You may have {condition}. Please consult a doctor.", "bot")
                self.restart_button.config(state=tk.NORMAL)
                return

        self.display_message("Your symptoms don't match any common conditions. Please consult a doctor.", "bot")
        self.restart_button.config(state=tk.NORMAL)

    def restart(self):
        """Restart the chatbot to ask questions again."""
        self.current_question = 0
        self.symptoms = {}
        for widget in self.scrollable_frame.winfo_children():
            widget.destroy()
        self.display_message("Restarting the diagnosis...\nPlease answer the following questions.", "bot")
        self.ask_question()
        self.restart_button.config(state=tk.DISABLED)


if __name__ == "__main__":
    root = tk.Tk()
    chatbot = MedicalExpertSystemChatbot(root)
    root.mainloop()
